# Consulta Postgres

In [ ]:
#!pip install pandas
#!pip install nltk
#!pip install psycopg2
#!pip install sqlalchemy
#!pip install flask

In [1]:
# imports iniciais
import os
import nltk
import pandas as pd
import pandas.io.sql as psql
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import math
from collections import Counter
from unicodedata import normalize
from nltk import ngrams
from flask import Flask
from flask import request
from flask import jsonify
from flask_cors import CORS
from nltk.stem import SnowballStemmer
import json
from pandas.io.json import json_normalize
from datetime import datetime


In [2]:
df = pd.read_csv('banco.csv')

In [3]:
print(df['id'].max()+1)

5


In [4]:
df.head()

,descricaoCadastroPartes,nomeParteReclamante,cpfParteReclamante,cpfCnpjParteReclamada,dataAjuizamentoInicial,dataInicioContrato,feriasVencidas,decimoTerceiroVencido,mesesFgtsNaoDepositado,multaRescisoriaFgtsDepositado,...,existenciaVinculoEmpregaticio,carteiraDevidamenteAnotada,funcao,motivoRescisao,jornadaSemanal,justicaGratuita,pedidos,id,acordo,link
0,NaN,José Reclamante da Silva,281.690.010-05,26.965.944/0001-97,2019-09-13,2014-03-20,NaN,NaN,NaN,0,...,NaN,NaN,Estivador,NaN,44,NaN,"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13},...",1,S,http://www.trt12.jus.br/busca/sentencas/browse...
1,NaN,José Reclamante da Silva,281.690.010-05,26.965.944/0001-97,2019-09-13,2014-03-20,NaN,NaN,NaN,0,...,NaN,NaN,Estivador,NaN,44,NaN,"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13},...",2,S,http://www.trt12.jus.br/busca/sentencas/browse...
2,NaN,José Reclamante da Silva,281.690.010-05,26.965.944/0001-97,2019-09-13,2014-03-20,NaN,NaN,NaN,0,...,NaN,NaN,Estivador,NaN,44,NaN,"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13},...",3,S,http://www.trt12.jus.br/busca/sentencas/browse...
3,NaN,José Reclamante da Silva,281.690.010-05,26.965.944/0001-97,2019-09-13,2014-03-20,NaN,NaN,NaN,0,...,NaN,NaN,Estivador,NaN,44,NaN,"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13},...",4,S,http://www.trt12.jus.br/busca/sentencas/browse...


In [5]:
df.set_index('id')

,descricaoCadastroPartes,nomeParteReclamante,cpfParteReclamante,cpfCnpjParteReclamada,dataAjuizamentoInicial,dataInicioContrato,feriasVencidas,decimoTerceiroVencido,mesesFgtsNaoDepositado,multaRescisoriaFgtsDepositado,...,meses13SalarioProporcional,existenciaVinculoEmpregaticio,carteiraDevidamenteAnotada,funcao,motivoRescisao,jornadaSemanal,justicaGratuita,pedidos,acordo,link
id,,,,,,,,,,,,,,,,,,,,,
1,NaN,José Reclamante da Silva,281.690.010-05,26.965.944/0001-97,2019-09-13,2014-03-20,NaN,NaN,NaN,0,...,0.5,NaN,NaN,Estivador,NaN,44,NaN,"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13},...",S,http://www.trt12.jus.br/busca/sentencas/browse...
2,NaN,José Reclamante da Silva,281.690.010-05,26.965.944/0001-97,2019-09-13,2014-03-20,NaN,NaN,NaN,0,...,0.5,NaN,NaN,Estivador,NaN,44,NaN,"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13},...",S,http://www.trt12.jus.br/busca/sentencas/browse...
3,NaN,José Reclamante da Silva,281.690.010-05,26.965.944/0001-97,2019-09-13,2014-03-20,NaN,NaN,NaN,0,...,0.5,NaN,NaN,Estivador,NaN,44,NaN,"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13},...",S,http://www.trt12.jus.br/busca/sentencas/browse...
4,NaN,José Reclamante da Silva,281.690.010-05,26.965.944/0001-97,2019-09-13,2014-03-20,NaN,NaN,NaN,0,...,0.5,NaN,NaN,Estivador,NaN,44,NaN,"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13},...",S,http://www.trt12.jus.br/busca/sentencas/browse...


In [66]:
DATABASE = 'banco.csv'
df_existente = pd.read_csv(DATABASE,header=0)

In [67]:
df_processo = df_existente.loc[df_existente['id'] == 1]

In [68]:
df_processo.head()

,descricaoCadastroPartes,nomeParteReclamante,cpfParteReclamante,cpfCnpjParteReclamada,dataAjuizamentoInicial,dataInicioContrato,feriasVencidas,decimoTerceiroVencido,mesesFgtsNaoDepositado,multaRescisoriaFgtsDepositado,...,existenciaVinculoEmpregaticio,carteiraDevidamenteAnotada,funcao,motivoRescisao,jornadaSemanal,justicaGratuita,pedidos,id,acordo,link
0,NaN,José Reclamante da Silva,281.690.010-05,26.965.944/0001-97,2019-09-13,2014-03-20,NaN,NaN,NaN,0,...,NaN,NaN,Estivador,NaN,44,NaN,"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13},...",1,S,http://www.trt12.jus.br/busca/sentencas/browse...


In [69]:
print(df_processo.to_json(orient='records'))

[{"descricaoCadastroPartes":null,"nomeParteReclamante":"Jos\u00e9 Reclamante da Silva","cpfParteReclamante":"281.690.010-05","cpfCnpjParteReclamada":"26.965.944\/0001-97","dataAjuizamentoInicial":"2019-09-13","dataInicioContrato":"2014-03-20","feriasVencidas":null,"decimoTerceiroVencido":null,"mesesFgtsNaoDepositado":null,"multaRescisoriaFgtsDepositado":0,"nomeParteReclamada":"Companhia Fict\u00edcia","dataTerminoContrato":"2015-04-22","salario":1788.38,"diasAvisoPrevio":13,"meses13SalarioProporcional":0.5,"existenciaVinculoEmpregaticio":null,"carteiraDevidamenteAnotada":null,"funcao":"Estivador","motivoRescisao":null,"jornadaSemanal":44,"justicaGratuita":null,"pedidos":"[{'tipo': 'AVISO_PREVIO', 'valor_pedido': 13}, {'tipo': 'DECIMO_TERC_PROPORCIONAL', 'valor_pedido': 0.5}]","id":1,"acordo":"S","link":"http:\/\/www.trt12.jus.br\/busca\/sentencas\/browse?q=aviso+pr%C3%A9vio&from=&to=&fq=&fq=ds_orgao_julgador%3A%221%C2%AA+VARA+DO+TRABALHO+DE+BLUMENAU%22"}]


In [87]:
import ast
from datetime import datetime
for i, p in df_existente.iterrows():
    print(p['acordo'])
    # Comparacao dos Nomes das Partes Reclamadas
    nomes_reclamante_parecidos = p['nomeParteReclamante'] == df_processo['nomeParteReclamante'].iloc[0]
    print('Nomes reclamante parecidos: '+str(nomes_reclamante_parecidos))
    # Comparacao dos Nomes das Partes Reclamantes
    nomes_reclamada_parecidos = p['nomeParteReclamada'] == df_processo['nomeParteReclamada'].iloc[0]
    print('Nomes parecidos: '+str(nomes_reclamada_parecidos))
    # Comapracao do prazo de Ajuizamento
    d1 = datetime.strptime(p['dataAjuizamentoInicial'], '%Y-%m-%d').date()
    d2 = datetime.strptime(df_processo['dataAjuizamentoInicial'].iloc[0], '%Y-%m-%d').date()
    dif_dataAjuizamentoInicial = abs((d2 - d1).days)
    print("Diferença Datas de Ajuizamento: "+str(dif_dataAjuizamentoInicial))
    # Calculo de dias de contratacao
    d1 = datetime.strptime(p['dataInicioContrato'], '%Y-%m-%d').date()
    d2 = datetime.strptime(df_processo['dataTerminoContrato'].iloc[0], '%Y-%m-%d').date()
    dif_meses = abs((d2 - d1).days)/30
    print("Período trabalhado em meses: "+str(dif_dataAjuizamentoInicial))
    propMeses13P1 = float(p['meses13SalarioProporcional'])
    propMeses13P2 = float(df_processo['meses13SalarioProporcional'].iloc[0])
    dif_propMeses13P = propMeses13P1-propMeses13P2
    print("Diferença da proporção de meses: %.2f" % dif_dataAjuizamentoInicial)
    salario1 = float(p['salario'])
    salario2 = float(df_processo['salario'].iloc[0])
    dif_salario = salario1-salario2
    print("Diferença do salário: %.2f" % dif_salario)
    print(df_processo['acordo'].iloc[0])
    pedidos = df_processo['pedidos'].iloc[0]
    x = ast.literal_eval(pedidos)
    pedido_concatenado = ''
    for pedido in x:
        pedido_concatenado += pedido['tipo']+"_"
    print(pedido_concatenado)

S
Nomes reclamante parecidos: True
Nomes parecidos: True
Diferença Datas de Ajuizamento: 0
Período trabalhado em meses: 0
Diferença da proporção de meses: 0.00
Diferença do salário: 0.00
S
AVISO_PREVIO_DECIMO_TERC_PROPORCIONAL_
S
Nomes reclamante parecidos: True
Nomes parecidos: True
Diferença Datas de Ajuizamento: 2
Período trabalhado em meses: 2
Diferença da proporção de meses: 2.00
Diferença do salário: 0.00
S
AVISO_PREVIO_DECIMO_TERC_PROPORCIONAL_
S
Nomes reclamante parecidos: True
Nomes parecidos: True
Diferença Datas de Ajuizamento: 2
Período trabalhado em meses: 2
Diferença da proporção de meses: 2.00
Diferença do salário: 0.00
S
AVISO_PREVIO_DECIMO_TERC_PROPORCIONAL_
S
Nomes reclamante parecidos: True
Nomes parecidos: True
Diferença Datas de Ajuizamento: 2
Período trabalhado em meses: 2
Diferença da proporção de meses: 2.00
Diferença do salário: 0.00
S
AVISO_PREVIO_DECIMO_TERC_PROPORCIONAL_


# Assunto

In [ ]:
df = pd.read_csv('dados_teste.csv',delimiter='|')

In [ ]:
df.describe()

In [ ]:
df.count()

In [ ]:
df.head()

In [ ]:
import string
from nltk.corpus import stopwords
portuguese_stops = set(stopwords.words('portuguese'))
ignore = ['@', '.', '!','?',',','$','-','\'s','g','(',')','[',']','``',':','http','html','//members']
#from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
#from nltk.stem import RegexpStemmer
from nltk.stem import SnowballStemmer
# Cria o Stemmer
stemmer = SnowballStemmer('english')
def preprocess_text(content):
    content.translate(str.maketrans('', '', string.punctuation))
    word_tokens = nltk.word_tokenize(content)
    #print(len(word_tokens),word_tokens)
    word_tokens2 = [w.lower() for w in word_tokens if w not in portuguese_stops]
    #print(len(word_tokens2),word_tokens2)
    word_tokens3 = [w for w in word_tokens2 if w not in ignore]
    #print(len(word_tokens3),word_tokens3)
    word_tokens4 = [stemmer.stem(w) for w in word_tokens3]
    #print(len(word_tokens4),word_tokens4)
    #return word_tokens4
    return ' '.join(word_tokens4)

In [ ]:
teste = 'Teste de sentença para tokenizar'

In [ ]:
preprocess_text(teste)

In [ ]:
df['nome'] = df['nome'].apply(preprocess_text)

In [ ]:
#!pip install gensim
#!pip install sklearn
#!pip install jieba

In [ ]:
from gensim import corpora, models, similarities

In [ ]:
texts = df['nome']
print(nomes)

In [ ]:
import nltk 
import re
import math

import pandas as pd

from collections import Counter
from unicodedata import normalize
from nltk import ngrams

#Regex para encontrar tokens
REGEX_WORD = re.compile(r'\w+')
#Numero de tokens em sequencia
N_GRAM_TOKEN = 3

#Faz a normalizacao do texto removendo espacos a mais e tirando acentos
def text_normalizer(src):
    return re.sub('\s+', ' ',
                normalize('NFKD', src)
                   .encode('ASCII','ignore')
                   .decode('ASCII')
           ).lower().strip()

#Faz o calculo de similaridade baseada no coseno
def cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        coef = float(numerator) / denominator
        if coef > 1:
            coef = 1
        return coef

#Monta o vetor de frequencia da sentenca
def sentence_to_vector(text, use_text_bigram):
    words = REGEX_WORD.findall(text)
    accumulator = []
    for n in range(1,N_GRAM_TOKEN):
        gramas = ngrams(words, n)
        for grama in gramas:
            accumulator.append(str(grama))

    if use_text_bigram:
        pairs = get_text_bigrams(text)
        for pair in pairs:
            accumulator.append(pair)

    return Counter(accumulator)

#Obtem a similaridade entre duas sentencas
def get_sentence_similarity(sentence1, sentence2, use_text_bigram=False):
    vector1 = sentence_to_vector(text_normalizer(sentence1), use_text_bigram)
    vector2 = sentence_to_vector(text_normalizer(sentence2), use_text_bigram)
    return cosine_similarity(vector1, vector2)

#Metodo de gerar bigramas de uma string
def get_text_bigrams(src):
    s = src.lower()
    return [s[i:i+2] for i in range(len(s) - 1)]

if __name__ == "__main__":
    w1 = 'COMERCIAL CASA DOS FRIOS - USAR LICINIO DIAS'
    words = [
        'ARES DOS ANDES - EXPORTACAO & IMPORTACAO LTDA', 
        'ADEGA DOS TRES IMPORTADORA', 
        'BODEGAS DE LOS ANDES COMERCIO DE VINHOS LTDA', 
        'ALL WINE IMPORTADORA'
    ]

    print('Busca: ' + w1)

    #Nivel de aceite (40%)
    cutoff = 0.40
    #Sentenças similares
    result = []

    for w2 in words:
        print('\nCosine Sentence --- ' + w2)

        #Calculo usando similaridade do coseno com apenas tokens
        similarity_sentence = get_sentence_similarity(w1, w2)
        print('\tSimilarity sentence: ' + str(similarity_sentence))

        #Calculo usando similaridade do coseno com tokens e com ngramas do texto
        similarity_sentence_text_bigram = get_sentence_similarity(w1, w2, use_text_bigram=True)
        print('\tSimilarity sentence: ' + str(similarity_sentence_text_bigram))

        if similarity_sentence >= cutoff:
            result.append((w2, similarity_sentence))

    print('\nResultado:')
    #Exibe resultados
    for data in result:
        print(data)

In [ ]:
texts.count()

In [ ]:
max_sim = 0
most_similar = ''
test = 'John da silva'
for w in texts:
    sim = get_sentence_similarity(test, w, use_text_bigram=True)
    if sim > max_sim:
        max_sim = sim
        most_similar = w
        print(str(sim)+' '+w)
print ('Mais similar a ' + test + ' = ' + most_similar)

In [ ]:
!pip install spacy

In [ ]:
import spacy
nlp = spacy.load('pt')

In [ ]:
import spacy
nlp = spacy.load('pt')
 
doc1 = nlp(u'Hello this is document similarity calculation')
doc2 = nlp(u'Hello this is python similarity calculation')
doc3 = nlp(u'Hi there')
 
print (doc1.similarity(doc2)) 
print (doc2.similarity(doc3)) 
print (doc1.similarity(doc3))  
 
max_sim = 0
most_similar = ''
test = 'John da silva'
for w in texts:
    sim = get_sentence_similarity(test, w, use_text_bigram=True)
    if sim > max_sim:
        max_sim = sim
        most_similar = w
        print(str(sim)+' '+w)
print ('Mais similar a ' + test + ' = ' + most_similar)

In [ ]:
!pip install flask

In [ ]:
pedido1 = {"descricaoCadastroPartes":null,"nomeParteReclamante":"José Reclamante da Silva","cpfParteReclamante":"281.690.010-05","cpfCnpjParteReclamada":"26.965.944/0001-97","dataAjuizamentoInicial":"2019-09-13","dataInicioContrato":"2014-03-20","feriasVencidas":null,"decimoTerceiroVencido":null,"mesesFgtsNaoDepositado":null,"multaRescisoriaFgtsDepositado":0,"nomeParteReclamada":"Companhia Fictícia","dataTerminoContrato":"2015-04-22","salario":1788.38,"diasAvisoPrevio":13,"meses13SalarioProporcional":0.5,"existenciaVinculoEmpregaticio":null,"carteiraDevidamenteAnotada":null,"funcao":"Estivador","motivoRescisao":null,"jornadaSemanal":44,"justicaGratuita":null,"pedidos":[{"tipo":"AVISO_PREVIO","valor_pedido":13},{"tipo":"DECIMO_TERC_PROPORCIONAL","valor_pedido":0.5}]}